In [2]:
!pip install icrawler


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os
import shutil
from icrawler.builtin import GoogleImageCrawler

# ==========================================
# 1. 데이터 리스트 (이름, 소속)
#    - 폴더명은 코드에서 자동 생성하므로 제거했습니다.
#    - 순서는 유지해야 카테고리별 로직(배우 등)이 적용됩니다.
# ==========================================
people_data = [
    # --- 1. 걸그룹 (1~15) ---
    ("제니", "블랙핑크"), ("카리나", "에스파"), ("장원영", "아이브"), ("민지", "뉴진스"), ("나연", "트와이스"),
    ("김채원", "르세라핌"), ("소연", "여자아이들"), ("아이린", "레드벨벳"), ("예지", "ITZY"), ("태연", "소녀시대"),
    ("설윤", "엔믹스"), ("시은", "스테이씨"), ("화사", "마마무"), ("아린", "오마이걸"), ("나띠", "키스오브라이프"),

    # --- 2. 보이그룹 (16~30) ---
    ("뷔", "방탄소년단"), ("민규", "세븐틴"), ("차은우", "아스트로"), ("백현", "엑소"), ("재현", "NCT 127"),
    ("현진", "스트레이키즈"), ("연준", "투모로우바이투게더"), ("지드래곤", "빅뱅"), ("주연", "더보이즈"), ("성한빈", "제로베이스원"),
    ("원빈", "라이즈"), ("키", "샤이니"), ("육성재", "비투비"), ("신유", "TWS"), ("지코", "블락비"),

    # --- 3. 배우/예능 (31~45) ---
    ("이정재", "오징어게임"), ("송혜교", "더글로리"), ("마동석", "범죄도시"), ("유재석", "런닝맨"), ("공유", "도깨비"),
    ("박서준", "이태원클라쓰"), ("전현무", "나혼자산다"), ("김수현", "눈물의여왕"), ("혜리", "응답하라 1988"), ("조정석", "슬기로운 의사생활"),
    ("고윤정", "무빙"), ("변우석", "선재 업고 튀어"), ("남궁민", "스토브리그"), ("김태리", "아가씨"), ("최민식", "파묘"),

    # --- 4. 스포츠 (46~50) ---
    ("손흥민", "토트넘"), ("페이커", "T1"), ("이정후", "샌프란시스코 자이언츠"), ("김연경", "흥국생명"), ("김예지", "사격 국가대표"),
]

# ==========================================
# 2. 크롤링 함수
# ==========================================
def create_dataset(base_dir="Dataset"):
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    # enumerate를 사용하여 인덱스(i)를 1부터 시작 (person01, person02...)
    for i, (name, group) in enumerate(people_data, start=1):
        
        # 폴더명 자동 생성: person01, person02 ... person50
        folder_name = f"person{i:02d}" 
        person_dir = os.path.join(base_dir, folder_name)
        
        if not os.path.exists(person_dir):
             os.makedirs(person_dir, exist_ok=True)
             print(f"\n[{folder_name}] Processing: {name} ({group})...")
        else:
            print(f"[{folder_name}] Skipping (Folder exists)")
            continue

        # ---------------------------------------------------------
        # Step A: 개인 사진 (Reference)
        # ---------------------------------------------------------
        temp_dir_ref = os.path.join(person_dir, "temp_ref")
        os.makedirs(temp_dir_ref, exist_ok=True) # 임시 폴더 강제 생성

        google_crawler = GoogleImageCrawler(storage={'root_dir': temp_dir_ref})
        ref_keyword = f"{name} 고화질 화보 독사진"
        
        # 크롤링 실행
        google_crawler.crawl(keyword=ref_keyword, max_num=3)

        # 파일 이동 (ref_01.jpg ...)
        downloaded_refs = sorted(os.listdir(temp_dir_ref))
        for idx, filename in enumerate(downloaded_refs):
            if idx >= 3: break
            src = os.path.join(temp_dir_ref, filename)
            dst = os.path.join(person_dir, f"ref_{idx+1:02d}.jpg") # ref_01.jpg 포맷
            shutil.move(src, dst)
        
        # 임시 폴더 삭제
        if os.path.exists(temp_dir_ref):
            os.rmdir(temp_dir_ref)

        # ---------------------------------------------------------
        # Step B: 단체 사진 (Test)
        # ---------------------------------------------------------
        temp_dir_group = os.path.join(person_dir, "temp_group")
        os.makedirs(temp_dir_group, exist_ok=True) # 임시 폴더 강제 생성

        google_crawler = GoogleImageCrawler(storage={'root_dir': temp_dir_group})
        
        group_keyword = f"{group} 단체사진"
        # 배우 카테고리(31~45번)는 '제작발표회' 키워드 추가
        if 31 <= i <= 45: 
             group_keyword = f"{group} 출연진 제작발표회"

        # 크롤링 실행
        google_crawler.crawl(keyword=group_keyword, max_num=1)

        # 파일 이동 (test_group.jpg)
        downloaded_groups = sorted(os.listdir(temp_dir_group))
        if downloaded_groups:
            src = os.path.join(temp_dir_group, downloaded_groups[0])
            dst = os.path.join(person_dir, "test_group.jpg")
            shutil.move(src, dst)
        
        # 임시 폴더 삭제
        if os.path.exists(temp_dir_group):
            os.rmdir(temp_dir_group)
        
        print(f"Finished: {folder_name} ({name})")

if __name__ == "__main__":
    create_dataset()
    print("\n모든 데이터셋 구축 완료!")

2025-12-02 15:30:58,823 - INFO - icrawler.crawler - start crawling...
2025-12-02 15:30:58,824 - INFO - icrawler.crawler - starting 1 feeder threads...
2025-12-02 15:30:58,824 - INFO - feeder - thread feeder-001 exit
2025-12-02 15:30:58,825 - INFO - icrawler.crawler - starting 1 parser threads...
2025-12-02 15:30:58,826 - INFO - icrawler.crawler - starting 1 downloader threads...



[person01] Processing: 제니 (블랙핑크)...


2025-12-02 15:30:59,204 - INFO - parser - parsing result page https://www.google.com/search?q=%EC%A0%9C%EB%8B%88+%EA%B3%A0%ED%99%94%EC%A7%88+%ED%99%94%EB%B3%B4+%EB%8F%85%EC%82%AC%EC%A7%84&ijn=0&start=0&tbs=&tbm=isch
Exception in thread parser-001:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ku/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/ku/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/anaconda3/envs/ku/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/ku/lib/python3.11/site-packages/icrawler/parser.py", line 93, in worker_exec
    for task in self.parse(response, **kwargs):
TypeError: 'NoneType' object is not iterable
2025-12-02 15:31:03,829 - INFO - downloader - no more download task for thread downloader-001
2025-12-02 15:31:03,831 - INFO - downloader - thread 

Finished: person01 (제니)

[person02] Processing: 카리나 (에스파)...


2025-12-02 15:31:09,382 - INFO - parser - parsing result page https://www.google.com/search?q=%EC%B9%B4%EB%A6%AC%EB%82%98+%EA%B3%A0%ED%99%94%EC%A7%88+%ED%99%94%EB%B3%B4+%EB%8F%85%EC%82%AC%EC%A7%84&ijn=0&start=0&tbs=&tbm=isch
Exception in thread parser-001:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ku/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/ku/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/anaconda3/envs/ku/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/ku/lib/python3.11/site-packages/icrawler/parser.py", line 93, in worker_exec
    for task in self.parse(response, **kwargs):
TypeError: 'NoneType' object is not iterable
2025-12-02 15:31:13,883 - INFO - downloader - no more download task for thread downloader-001
2025-12-02 15:31:13,886 - INFO - downloader 

...

2025-12-02 15:32:19,655 - INFO - parser - parsing result page https://www.google.com/search?q=%EC%98%88%EC%A7%80+%EA%B3%A0%ED%99%94%EC%A7%88+%ED%99%94%EB%B3%B4+%EB%8F%85%EC%82%AC%EC%A7%84&ijn=0&start=0&tbs=&tbm=isch
Exception in thread parser-001:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ku/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/ku/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/anaconda3/envs/ku/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/ku/lib/python3.11/site-packages/icrawler/parser.py", line 93, in worker_exec
    for task in self.parse(response, **kwargs):
TypeError: 'NoneType' object is not iterable
2025-12-02 15:32:24,257 - INFO - downloader - no more download task for thread downloader-001
2025-12-02 15:32:24,260 - INFO - downloader - thread 

Finished: person09 (예지)

[person10] Processing: 태연 (소녀시대)...


2025-12-02 15:32:29,723 - INFO - parser - parsing result page https://www.google.com/search?q=%ED%83%9C%EC%97%B0+%EA%B3%A0%ED%99%94%EC%A7%88+%ED%99%94%EB%B3%B4+%EB%8F%85%EC%82%AC%EC%A7%84&ijn=0&start=0&tbs=&tbm=isch
Exception in thread parser-001:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ku/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/ku/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/anaconda3/envs/ku/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/ku/lib/python3.11/site-packages/icrawler/parser.py", line 93, in worker_exec
    for task in self.parse(response, **kwargs):
TypeError: 'NoneType' object is not iterable


KeyboardInterrupt: 

2025-12-02 15:32:34,294 - INFO - downloader - no more download task for thread downloader-001
2025-12-02 15:32:34,295 - INFO - downloader - thread downloader-001 exit


In [ ]:
import os
import shutil
# Google 대신 Bing 크롤러를 임포트합니다.
from icrawler.builtin import BingImageCrawler

# ==========================================
# 1. 데이터 리스트 (이름, 소속)
# ==========================================
people_data = [
    # --- 1. 걸그룹 (1~15) ---
    ("제니", "블랙핑크"), ("카리나", "에스파"), ("장원영", "아이브"), ("민지", "뉴진스"), ("나연", "트와이스"),
    ("김채원", "르세라핌"), ("소연", "여자아이들"), ("아이린", "레드벨벳"), ("예지", "ITZY"), ("태연", "소녀시대"),
    ("설윤", "엔믹스"), ("시은", "스테이씨"), ("화사", "마마무"), ("아린", "오마이걸"), ("나띠", "키스오브라이프"),

    # --- 2. 보이그룹 (16~30) ---
    ("뷔", "방탄소년단"), ("민규", "세븐틴"), ("차은우", "아스트로"), ("백현", "엑소"), ("재현", "NCT 127"),
    ("현진", "스트레이키즈"), ("연준", "투모로우바이투게더"), ("지드래곤", "빅뱅"), ("주연", "더보이즈"), ("성한빈", "제로베이스원"),
    ("원빈", "라이즈"), ("키", "샤이니"), ("육성재", "비투비"), ("신유", "TWS"), ("지코", "블락비"),

    # --- 3. 배우/예능 (31~45) ---
    ("이정재", "오징어게임"), ("송혜교", "더글로리"), ("마동석", "범죄도시"), ("유재석", "런닝맨"), ("공유", "도깨비"),
    ("박서준", "이태원클라쓰"), ("전현무", "나혼자산다"), ("김수현", "눈물의여왕"), ("혜리", "응답하라 1988"), ("조정석", "슬기로운 의사생활"),
    ("고윤정", "무빙"), ("변우석", "선재 업고 튀어"), ("남궁민", "스토브리그"), ("김태리", "아가씨"), ("최민식", "파묘"),

    # --- 4. 스포츠 (46~50) ---
    ("손흥민", "토트넘"), ("페이커", "T1"), ("이정후", "샌프란시스코 자이언츠"), ("김연경", "흥국생명"), ("김예지", "사격 국가대표"),
]

# ==========================================
# 2. 크롤링 함수
# ==========================================
def create_dataset(base_dir="Dataset"):
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    for i, (name, group) in enumerate(people_data, start=1):
        
        folder_name = f"person{i:02d}" 
        person_dir = os.path.join(base_dir, folder_name)
        
        # 폴더가 이미 있고, 내용물도 차 있으면 건너뛰기
        if os.path.exists(person_dir) and len(os.listdir(person_dir)) >= 4:
            print(f"[{folder_name}] Skipping (Already done)")
            continue

        if not os.path.exists(person_dir):
             os.makedirs(person_dir, exist_ok=True)
        
        print(f"\n[{folder_name}] Processing: {name} ({group})...")

        # ---------------------------------------------------------
        # Step A: 개인 사진 (Reference)
        # ---------------------------------------------------------
        temp_dir_ref = os.path.join(person_dir, "temp_ref")
        os.makedirs(temp_dir_ref, exist_ok=True) 

        # [변경점] GoogleImageCrawler -> BingImageCrawler
        crawler = BingImageCrawler(storage={'root_dir': temp_dir_ref})
        ref_keyword = f"{name} 고화질 화보 독사진"
        
        # Bing은 필터 옵션이 강력해서, 사람 얼굴 위주로 찾도록 설정할 수도 있습니다.
        # 여기선 기본 검색으로 진행합니다.
        crawler.crawl(keyword=ref_keyword, max_num=3)

        # 파일 이동
        downloaded_refs = sorted(os.listdir(temp_dir_ref))
        for idx, filename in enumerate(downloaded_refs):
            if idx >= 3: break
            src = os.path.join(temp_dir_ref, filename)
            dst = os.path.join(person_dir, f"ref_{idx+1:02d}.jpg")
            shutil.move(src, dst)
        
        # 임시 폴더 정리
        if os.path.exists(temp_dir_ref):
            for file in os.listdir(temp_dir_ref): # 남은 파일 삭제
                os.remove(os.path.join(temp_dir_ref, file))
            os.rmdir(temp_dir_ref)

        # ---------------------------------------------------------
        # Step B: 단체 사진 (Test)
        # ---------------------------------------------------------
        temp_dir_group = os.path.join(person_dir, "temp_group")
        os.makedirs(temp_dir_group, exist_ok=True)

        # [변경점] GoogleImageCrawler -> BingImageCrawler
        crawler = BingImageCrawler(storage={'root_dir': temp_dir_group})
        
        group_keyword = f"{group} 단체사진"
        if 31 <= i <= 45: 
             group_keyword = f"{group} 출연진 제작발표회"

        crawler.crawl(keyword=group_keyword, max_num=1)

        # 파일 이동
        downloaded_groups = sorted(os.listdir(temp_dir_group))
        if downloaded_groups:
            src = os.path.join(temp_dir_group, downloaded_groups[0])
            dst = os.path.join(person_dir, "test_group.jpg")
            shutil.move(src, dst)
        
        # 임시 폴더 정리
        if os.path.exists(temp_dir_group):
            for file in os.listdir(temp_dir_group):
                os.remove(os.path.join(temp_dir_group, file))
            os.rmdir(temp_dir_group)
        
        print(f"Finished: {folder_name} ({name})")

if __name__ == "__main__":
    create_dataset()
    print("\n모든 데이터셋 구축 완료!")

2025-12-02 15:33:03,983 - INFO - icrawler.crawler - start crawling...
2025-12-02 15:33:03,983 - INFO - icrawler.crawler - starting 1 feeder threads...
2025-12-02 15:33:03,984 - INFO - feeder - thread feeder-001 exit
2025-12-02 15:33:03,984 - INFO - icrawler.crawler - starting 1 parser threads...
2025-12-02 15:33:03,985 - INFO - icrawler.crawler - starting 1 downloader threads...
2025-12-02 15:33:04,162 - INFO - parser - parsing result page https://www.bing.com/images/async?q=제니 고화질 화보 독사진&first=0



[person01] Processing: 제니 (블랙핑크)...


2025-12-02 15:33:04,350 - INFO - downloader - image #1	https://www.mhnse.com/news/photo/202210/146376_139561_3019.jpg
2025-12-02 15:33:06,005 - INFO - downloader - image #2	https://files.vogue.co.th/uploads/galleryforjenlisa_553010826_p_3093418783419213488_1_p_3093418783419213488.jpg
2025-12-02 15:33:06,732 - INFO - downloader - image #3	https://image.librewiki.net/0/09/Sd_jen.jpg
2025-12-02 15:33:07,481 - INFO - downloader - downloaded images reach max num, thread downloader-001 is ready to exit
2025-12-02 15:33:07,483 - INFO - downloader - thread downloader-001 exit
2025-12-02 15:33:08,003 - INFO - icrawler.crawler - Crawling task done!
2025-12-02 15:33:08,007 - INFO - icrawler.crawler - start crawling...
2025-12-02 15:33:08,008 - INFO - icrawler.crawler - starting 1 feeder threads...
2025-12-02 15:33:08,010 - INFO - feeder - thread feeder-001 exit
2025-12-02 15:33:08,011 - INFO - icrawler.crawler - starting 1 parser threads...
2025-12-02 15:33:08,013 - INFO - icrawler.crawler - star

Finished: person01 (제니)

[person02] Processing: 카리나 (에스파)...


2025-12-02 15:33:10,332 - INFO - downloader - image #1	https://img.sbs.co.kr/newsnet/etv/upload/2020/10/28/30000654805_1280.jpg
2025-12-02 15:33:10,504 - INFO - downloader - image #2	https://cdn.mhnse.com/news/photo/202208/134498_126052_538.jpg
2025-12-02 15:33:10,600 - INFO - downloader - image #3	https://news.nateimg.co.kr/orgImg/su/2022/12/16/572390_797883_3250.jpg
2025-12-02 15:33:10,960 - INFO - downloader - downloaded images reach max num, thread downloader-001 is ready to exit
2025-12-02 15:33:10,961 - INFO - downloader - thread downloader-001 exit
2025-12-02 15:33:11,036 - INFO - icrawler.crawler - Crawling task done!
2025-12-02 15:33:11,041 - INFO - icrawler.crawler - start crawling...
2025-12-02 15:33:11,042 - INFO - icrawler.crawler - starting 1 feeder threads...
2025-12-02 15:33:11,043 - INFO - feeder - thread feeder-001 exit
2025-12-02 15:33:11,043 - INFO - icrawler.crawler - starting 1 parser threads...
2025-12-02 15:33:11,045 - INFO - icrawler.crawler - starting 1 downlo

...

2025-12-02 15:35:59,719 - INFO - parser - parsing result page https://www.bing.com/images/async?q=김연경 고화질 화보 독사진&first=0
2025-12-02 15:35:59,869 - INFO - downloader - image #1	https://t1.daumcdn.net/news/202103/31/kbs/20210331135756673wvrs.jpg
2025-12-02 15:36:00,115 - INFO - downloader - image #2	https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https:%2F%2Fblog.kakaocdn.net%2Fdn%2FbH4Ecm%2FbtseyzgKW9H%2FrnQDCaNjKx3XaJRydr92p1%2Fimg.jpg
2025-12-02 15:36:00,212 - INFO - downloader - image #3	https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https:%2F%2Fblog.kakaocdn.net%2Fdn%2Fmq7F2%2Fbtr6Qplv8MU%2FzKnnbzOHU1ViVWpkw57ZaK%2Fimg.jpg
2025-12-02 15:36:00,319 - INFO - downloader - downloaded images reach max num, thread downloader-001 is ready to exit
2025-12-02 15:36:00,320 - INFO - downloader - thread downloader-001 exit
2025-12-02 15:36:00,520 - INFO - icrawler.crawler - Crawling task done!
2025-12-02 15:36:00,525 - INFO - icrawler.crawler - start crawling...
2025

Finished: person49 (김연경)

[person50] Processing: 김예지 (사격 국가대표)...


2025-12-02 15:36:01,943 - INFO - parser - downloaded image reached max num, thread parser-001 is ready to exit
2025-12-02 15:36:01,944 - INFO - parser - thread parser-001 exit
2025-12-02 15:36:02,284 - INFO - downloader - image #1	https://cdn.thefairnews.co.kr/news/photo/202401/20040_49820_4627.jpg
2025-12-02 15:36:02,530 - INFO - downloader - image #2	https://img.asiatoday.co.kr/file/2024y/01m/04d/2024010401000367300030261.jpg
2025-12-02 15:36:02,682 - INFO - downloader - image #3	https://www.bntnews.co.kr/data/bnt/image/2022/10/04/bnt202210040058.jpg
2025-12-02 15:36:03,038 - INFO - downloader - downloaded images reach max num, thread downloader-001 is ready to exit
2025-12-02 15:36:03,039 - INFO - downloader - thread downloader-001 exit
2025-12-02 15:36:03,548 - INFO - icrawler.crawler - Crawling task done!
2025-12-02 15:36:03,554 - INFO - icrawler.crawler - start crawling...
2025-12-02 15:36:03,554 - INFO - icrawler.crawler - starting 1 feeder threads...
2025-12-02 15:36:03,555 - I

Finished: person50 (김예지)

모든 데이터셋 구축 완료!


2025-12-02 15:36:05,137 - INFO - parser - downloaded image reached max num, thread parser-001 is ready to exit
2025-12-02 15:36:05,138 - INFO - parser - thread parser-001 exit
